In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# Antibody 쪽 결과 로딩
ANTIBODY_TRACE_PATH = Path("../artifact/core3/core3_instability_risk_with_trace.csv")

antibody_df = pd.read_csv(ANTIBODY_TRACE_PATH)

print("Antibody trace shape:", antibody_df.shape)
print("Columns:", antibody_df.columns.tolist())

antibody_df.head()

Antibody trace shape: (60, 9)
Columns: ['antibody_key', 'step', 'pred_score', 'rolling_mean_pred_score', 'rolling_std_pred_score', 'recent_mutation_count_k', 'cooldown_left', 'toggle_event', 'instability_risk']


,antibody_key,step,pred_score,rolling_mean_pred_score,rolling_std_pred_score,recent_mutation_count_k,cooldown_left,toggle_event,instability_risk
0,GDPa1-001,1,2.013943,NaN,NaN,NaN,0,1,0.925016
1,GDPa1-001,2,1.966444,NaN,NaN,NaN,0,1,0.925016
2,GDPa1-001,3,1.943947,1.974778,0.035734,1.0,0,0,0.165824
3,GDPa1-001,4,1.916268,1.942219,0.025133,0.0,0,0,0.486062
4,GDPa1-001,5,1.939915,1.933376,0.014953,1.0,0,1,0.636068


Antibody decision instability 지표 계산

정의 (Core 4 기준, 재정의 없음)
	•	toggle_event: 이미 Core 3에서 정의됨
	•	toggle_rate = toggle_event 평균
	•	burst_length = 연속 toggle 구간 길이

In [6]:
def compute_burst_lengths(toggle_series):
    bursts = []
    current = 0
    for v in toggle_series:
        if v == 1:
            current += 1
        else:
            if current > 0:
                bursts.append(current)
                current = 0
    if current > 0:
        bursts.append(current)
    return bursts

antibody_summary = []

for ab, g in antibody_df.groupby("antibody_key"):
    toggle_rate = g["toggle_event"].mean()
    bursts = compute_burst_lengths(g["toggle_event"].values)
    
    antibody_summary.append({
        "system": "antibody",
        "entity": ab,
        "toggle_rate": toggle_rate,
        "mean_burst_length": np.mean(bursts) if bursts else 0,
        "max_burst_length": np.max(bursts) if bursts else 0,
        "num_toggles": int(g["toggle_event"].sum())
    })

antibody_summary_df = pd.DataFrame(antibody_summary)
antibody_summary_df

,system,entity,toggle_rate,mean_burst_length,max_burst_length,num_toggles
0,antibody,GDPa1-001,0.50,1.428571,2,10
1,antibody,GDPa1-045,0.35,1.400000,2,7
2,antibody,GDPa1-183,0.40,2.000000,3,8


In [7]:
BATTERY_PATH = Path("../../data_csv/NASA_Battery_Degradation.csv")
battery_df = pd.read_csv(BATTERY_PATH)

print("Battery shape:", battery_df.shape)
print("Columns:", battery_df.columns.tolist())

battery_df.head()

Battery shape: (1415, 7)
Columns: ['battery_id', 'cycle', 'voltage', 'temperature', 'capacity', 'soh', 'rul']


,battery_id,cycle,voltage,temperature,capacity,soh,rul
0,B0005,1,3.532781,32.536891,1.861976,1.000000,167
1,B0005,2,3.542968,32.643595,1.851862,0.994568,166
2,B0005,3,3.553056,32.522526,1.840808,0.988631,165
3,B0005,4,3.545849,32.492083,1.850058,0.993599,164
4,B0005,5,3.544456,32.368612,1.849432,0.993263,163


Battery에서 ‘의사결정 이벤트’ 정의

⚠️ 핵심 포인트
	•	예측 기반 판단 ❌
	•	상태 기반 제어만 가정

여기서는 매우 보수적으로 정의한다:
	•	control_event = 1
→ SOH 감소가 일정 이상 “누적”될 때만
	•	단기 변동에는 반응하지 않음

In [8]:
# battery 1개만 고정 (cycle 가장 긴 것)
battery_id = (
    battery_df.groupby("battery_id")["cycle"]
    .nunique()
    .sort_values(ascending=False)
    .index[0]
)

one_battery = (
    battery_df[battery_df["battery_id"] == battery_id]
    .sort_values("cycle")
    .reset_index(drop=True)
)

one_battery[["cycle", "soh"]].head()

# 상태 기반 히스테리시스: 누적 SOH 감소만 본다
one_battery["soh_delta"] = one_battery["soh"].diff()
one_battery["soh_delta_cum"] = one_battery["soh_delta"].cumsum()

# 제어 이벤트 정의 (예시적, 구조 설명용)
STATE_THRESHOLD = -0.05  # 단기 아님, 누적 기준

one_battery["control_event"] = (
    one_battery["soh_delta_cum"] < STATE_THRESHOLD
).astype(int)

one_battery[["cycle", "soh", "soh_delta_cum", "control_event"]].head(15)

,cycle,soh,soh_delta_cum,control_event
0,1,1.000000,NaN,0
1,2,0.994568,-0.005432,0
2,3,0.988631,-0.011369,0
3,4,0.993599,-0.006401,0
4,5,0.993263,-0.006737,0
5,6,0.988808,-0.011192,0
6,7,0.988531,-0.011469,0
7,8,0.983452,-0.016548,0
8,9,0.987953,-0.012047,0
9,10,0.982837,-0.017163,0


In [9]:
battery_toggle_rate = one_battery["control_event"].mean()
battery_bursts = compute_burst_lengths(one_battery["control_event"].values)

battery_summary_df = pd.DataFrame([{
    "system": "battery",
    "entity": battery_id,
    "toggle_rate": battery_toggle_rate,
    "mean_burst_length": np.mean(battery_bursts) if battery_bursts else 0,
    "max_burst_length": np.max(battery_bursts) if battery_bursts else 0,
    "num_toggles": int(one_battery["control_event"].sum())
}])

battery_summary_df # Battery decision stability 지표 계산

,system,entity,toggle_rate,mean_burst_length,max_burst_length,num_toggles
0,battery,B0005,0.732143,41.0,118,123


In [10]:
comparison_df = pd.concat(
    [antibody_summary_df, battery_summary_df],
    ignore_index=True
)

comparison_df # Antibody vs Battery 숫자 비교표 (단위 같을 필요 없음. 구조 비교용 테이블)

,system,entity,toggle_rate,mean_burst_length,max_burst_length,num_toggles
0,antibody,GDPa1-001,0.500000,1.428571,2,10
1,antibody,GDPa1-045,0.350000,1.400000,2,7
2,antibody,GDPa1-183,0.400000,2.000000,3,8
3,battery,B0005,0.732143,41.000000,118,123


In [11]:
print("""
[Core 4-03 구조 비교 결론]

- 항체 시스템에서는 예측 변동이 직접 의사결정 트리거로 작동하여
  toggle_rate와 burst_length가 높게 나타난다.
- 배터리 시스템에서는 예측이 아니라 상태(SOH)가 판단 기준이며,
  상태 기반 히스테리시스로 인해 toggle이 거의 발생하지 않는다.
- 즉, 배터리는 '예측 → 즉각 판단' 구조가 아니라
  '상태 계측 → 완충 → 판단' 구조를 가진다.

→ 의사결정 안정성의 차이는 예측 성능이 아니라
  상태 계측 구조의 차이에서 발생한다.
""")


[Core 4-03 구조 비교 결론]

- 항체 시스템에서는 예측 변동이 직접 의사결정 트리거로 작동하여
  toggle_rate와 burst_length가 높게 나타난다.
- 배터리 시스템에서는 예측이 아니라 상태(SOH)가 판단 기준이며,
  상태 기반 히스테리시스로 인해 toggle이 거의 발생하지 않는다.
- 즉, 배터리는 '예측 → 즉각 판단' 구조가 아니라
  '상태 계측 → 완충 → 판단' 구조를 가진다.

→ 의사결정 안정성의 차이는 예측 성능이 아니라
  상태 계측 구조의 차이에서 발생한다.

